In [17]:
import numpy as np
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from lime.lime_tabular import LimeTabularExplainer

import argparse
import datetime
import random
import signal
import sys
import warnings
from collections import Counter
from functools import partial
from pathlib import Path
from types import FrameType
from typing import Any

import dill
import multiprocess
import numpy as np
import pandas as pd
import torch
from synth_xai.explanations.explanation_utils import (
    evaluate_bb,
    find_top_closest_rows,
    get_test_data,
    is_explainer_supported,
    label_synthetic_data,
    load_bb,
    load_synthetic_data,
    make_predictions,
    prepare_neighbours,
    setup_wandb,
    transform_input_data,
)
from loguru import logger
from multiprocess import Pool
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import (
    MinMaxScaler,
)

from synth_xai.bb_architectures import MultiClassModel, SimpleModel
from synth_xai.explanations.explainer_model import ExplainerModel
from synth_xai.utils import (
    prepare_adult,
    prepare_dutch,
    prepare_letter,
)

# Lore

In [18]:
from lore_sa.dataset import TabularDataset

# from lore_sa.lorem import LOREM

In [19]:
class MyModel:
    def __init__(self, model: torch.nn.Module, scaler) -> None:
        self.model = model
        self.scaler = scaler

    def predict(self, x: np.ndarray) -> torch.Tensor:

        x = self.scaler.transform(x)

        predictions = []
        for sample in x:
            sample = torch.Tensor(sample)
            predictions.append(self.model(sample).argmax().item())
        return np.array(predictions)
        # print(x)
        # x = torch.Tensor(x)
        # print(x)
        # return np.array([self.model(sample).argmax().item() for sample in x])
    

seed = 112
current_script_path = Path("./").resolve()
print(current_script_path)
x_train, _, x_test, _, _, _, train_df, test_data = prepare_adult(sweep=False, seed=seed, current_path=current_script_path)

# Get the feature names after one-hot encoding
feature_names = list(train_df.drop(columns=["income_binary"]).columns)

bb_path = "../../../artifacts/adult/bb/adult_BB.pth"
bb = load_bb(bb_path)

bb.to("cpu")


/home/lcorbucci/synth_xai/src/synth_xai/comparison


SimpleModel(
  (layer1): Linear(in_features=111, out_features=32, bias=True)
  (layer2): Linear(in_features=32, out_features=2, bias=True)
)

In [20]:
import copy 

train_df_tmp = copy.copy(train_df)
train_df_tmp = train_df_tmp.drop(columns=["income_binary"])

scaler = MinMaxScaler()
x_train = scaler.fit_transform(train_df_tmp)

In [21]:
model = MyModel(model=bb, scaler=scaler)

num_classes = 2

In [22]:
train_df["income_binary"] = [int(x) for x in train_df["income_binary"]]

In [23]:
train_df["income_binary"] = train_df["income_binary"].astype('category')

In [24]:
dataset = TabularDataset.from_dict(train_df, class_name = "income_binary")
dataset.df.dropna(inplace = True)
dataset.df

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,sex_binary,race_binary,age_binary,workclass_ ?,...,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia,income_binary
12555,46,122026,13,0,0,44,1,1,1,False,...,False,False,False,False,False,False,True,False,False,1
6963,50,173224,14,0,0,40,0,1,1,False,...,False,False,False,False,False,False,True,False,False,0
35772,23,398130,9,0,0,96,1,1,0,False,...,False,False,False,False,False,False,True,False,False,0
16995,37,136028,11,0,0,40,1,1,1,False,...,False,False,False,False,False,False,True,False,False,0
36079,48,196689,13,0,0,40,1,1,1,False,...,False,False,False,False,False,False,True,False,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37628,46,151107,10,0,1977,60,1,0,1,False,...,False,False,False,False,False,False,True,False,False,1
10381,38,212245,10,0,0,40,1,1,1,False,...,False,False,False,False,False,False,True,False,False,0
9646,62,26911,4,0,0,66,0,1,0,False,...,False,False,False,False,False,False,True,False,False,0
34324,45,302677,10,0,1340,50,0,1,1,False,...,False,False,False,False,False,False,True,False,False,0


In [25]:
dataset.descriptor.keys()

dict_keys(['numeric', 'categorical', 'ordinal', 'target'])

In [26]:
from lore_sa.bbox import sklearn_classifier_bbox

In [27]:
bbox = sklearn_classifier_bbox.sklearnBBox(model)

In [28]:
from lore_sa.lore import TabularRandomGeneratorLore

tabularLore = TabularRandomGeneratorLore(bbox, dataset)

In [29]:
num_row = 10
x = dataset.df.iloc[num_row][:-1] # we exclude the target feature
x

age                                    60
fnlwgt                             129379
education-num                          10
capital-gain                            0
capital-loss                            0
                                    ...  
native-country_ Thailand            False
native-country_ Trinadad&Tobago     False
native-country_ United-States        True
native-country_ Vietnam             False
native-country_ Yugoslavia          False
Name: 24447, Length: 111, dtype: object

In [30]:
# for i in range(0, 100):
#     x = dataset.df.iloc[i][:-1] # we exclude the target feature
#     print(bb(torch.Tensor(scaler.transform(np.array(x.values).reshape(1, -1)))).argmax())

In [31]:
num_rows = 100
multi_x = dataset.df.iloc[:num_row] # we exclude the target feature
multi_x = multi_x.drop(columns = ["income_binary"])

In [32]:
bb(torch.Tensor(scaler.transform(np.array(x.values).reshape(1, -1)))).argmax()

tensor(1)

In [33]:
model.predict(multi_x)

array([1, 1, 0, 0, 1, 0, 1, 0, 1, 0])

In [34]:
dataset.descriptor.keys()

dict_keys(['numeric', 'categorical', 'ordinal', 'target'])

In [37]:
dataset.descriptor["categorical"]

{'workclass_ ?': {'index': 9,
  'distinct_values': [False, True],
  'count': {False: 36868, True: 2204}},
 'workclass_ Federal-gov': {'index': 10,
  'distinct_values': [False, True],
  'count': {False: 37909, True: 1163}},
 'workclass_ Local-gov': {'index': 11,
  'distinct_values': [False, True],
  'count': {False: 36568, True: 2504}},
 'workclass_ Never-worked': {'index': 12,
  'distinct_values': [False, True],
  'count': {False: 39062, True: 10}},
 'workclass_ Private': {'index': 13,
  'distinct_values': [True, False],
  'count': {True: 27133, False: 11939}},
 'workclass_ Self-emp-inc': {'index': 14,
  'distinct_values': [False, True],
  'count': {False: 37723, True: 1349}},
 'workclass_ Self-emp-not-inc': {'index': 15,
  'distinct_values': [False, True],
  'count': {False: 35978, True: 3094}},
 'workclass_ State-gov': {'index': 16,
  'distinct_values': [False, True],
  'count': {False: 37475, True: 1597}},
 'workclass_ Without-pay': {'index': 17,
  'distinct_values': [False, True],


In [ ]:
dataset.descriptor["target"]

In [ ]:
# when
explanation = tabularLore.explain(x)
# then
print(explanation)

In [ ]:
for premise in explanation["rule"]['premises']:
    print(premise["attr"])

In [90]:
with open("/home/lcorbucci/synth_xai/artifacts/adult/comparison_explanation/lore/lore_1.pkl", "rb") as f:
    shap_explanation = dill.load(f)

In [ ]:
shap_explanation[1]

In [ ]:
shap_explanation[1]["rule"]["premises"]

# Lore - Dutch

In [2]:
from lore_sa.dataset import TabularDataset

# from lore_sa.lorem import LOREM

In [3]:
!export CUDA_VISIBLE_DEVICES=1

In [4]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"]="1"


In [5]:
class MyModel:
    def __init__(self, model: torch.nn.Module, scaler) -> None:
        self.model = model
        self.scaler = scaler

    def predict(self, x: np.ndarray) -> torch.Tensor:

        x = self.scaler.transform(x)

        predictions = []
        for sample in x:
            sample = torch.Tensor(sample)
            predictions.append(self.model(sample).argmax().item())
        return np.array(predictions)
        # print(x)
        # x = torch.Tensor(x)
        # print(x)
        # return np.array([self.model(sample).argmax().item() for sample in x])
    

seed = 112
current_script_path = Path("./").resolve()
print(current_script_path)
x_train, _, x_test, _, _, _, train_df, test_data = prepare_dutch(sweep=False, seed=seed, current_path=current_script_path)

# Get the feature names after one-hot encoding
feature_names = list(train_df.drop(columns=["occupation_binary"]).columns)

bb_path = "../../../artifacts/dutch/bb/dutch_BB.pth"
bb = load_bb(bb_path)

bb.to("cpu")

/home/lcorbucci/synth_xai/src/synth_xai/comparison


SimpleModel(
  (layer1): Linear(in_features=11, out_features=32, bias=True)
  (layer2): Linear(in_features=32, out_features=2, bias=True)
)

In [6]:
import copy 

train_df_tmp = copy.copy(train_df)
train_df_tmp = train_df_tmp.drop(columns=["occupation_binary"])

scaler = MinMaxScaler()
x_train = scaler.fit_transform(train_df_tmp)

In [7]:
model = MyModel(model=bb, scaler=scaler)

num_classes = 2

In [8]:
train_df["occupation_binary"] = [int(x) for x in train_df["occupation_binary"]]

In [9]:
train_df["occupation_binary"] = train_df["occupation_binary"].astype('category')    

In [10]:
dataset = TabularDataset.from_dict(train_df, class_name = "occupation_binary")
dataset.df.dropna(inplace = True)
dataset.df

,age,household_position,household_size,prev_residence_place,citizenship,country_birth,edu_level,economic_status,cur_eco_activity,Marital_status,sex_binary,occupation_binary
41656,8,1122,125,1,1,1,3,111,136,2,1,1
10920,6,1121,112,1,1,1,3,111,122,2,1,0
9932,10,1122,113,1,1,1,3,111,137,2,1,0
10634,6,1122,113,1,1,1,3,111,132,2,0,1
27636,6,1210,111,1,1,1,3,111,136,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
59169,5,1131,112,1,1,1,3,111,138,1,0,1
32180,11,1122,113,1,1,1,2,111,133,2,1,1
4499,8,1210,111,1,1,1,3,111,122,4,1,0
14382,8,1132,113,1,1,1,3,111,135,4,1,0


In [11]:
dataset.descriptor.keys()

dict_keys(['numeric', 'categorical', 'ordinal', 'target'])

In [38]:
dataset.descriptor["numeric"]

{'workclass_ ?': {'index': 9,
  'distinct_values': [False, True],
  'count': {False: 36868, True: 2204}},
 'workclass_ Federal-gov': {'index': 10,
  'distinct_values': [False, True],
  'count': {False: 37909, True: 1163}},
 'workclass_ Local-gov': {'index': 11,
  'distinct_values': [False, True],
  'count': {False: 36568, True: 2504}},
 'workclass_ Never-worked': {'index': 12,
  'distinct_values': [False, True],
  'count': {False: 39062, True: 10}},
 'workclass_ Private': {'index': 13,
  'distinct_values': [True, False],
  'count': {True: 27133, False: 11939}},
 'workclass_ Self-emp-inc': {'index': 14,
  'distinct_values': [False, True],
  'count': {False: 37723, True: 1349}},
 'workclass_ Self-emp-not-inc': {'index': 15,
  'distinct_values': [False, True],
  'count': {False: 35978, True: 3094}},
 'workclass_ State-gov': {'index': 16,
  'distinct_values': [False, True],
  'count': {False: 37475, True: 1597}},
 'workclass_ Without-pay': {'index': 17,
  'distinct_values': [False, True],


In [ ]:
from lore_sa.bbox import sklearn_classifier_bbox

In [ ]:
bbox = sklearn_classifier_bbox.sklearnBBox(model)

In [ ]:
from lore_sa.lore import TabularRandomGeneratorLore

tabularLore = TabularRandomGeneratorLore(bbox, dataset)

In [ ]:
num_row = 10
x = dataset.df.iloc[num_row][:-1] # we exclude the target feature
x

In [ ]:
# for i in range(0, 100):
#     x = dataset.df.iloc[i][:-1] # we exclude the target feature
#     print(bb(torch.Tensor(scaler.transform(np.array(x.values).reshape(1, -1)))).argmax())

In [ ]:
num_rows = 100
multi_x = dataset.df.iloc[:num_row] # we exclude the target feature
multi_x = multi_x.drop(columns = ["income_binary"])

In [ ]:
bb(torch.Tensor(scaler.transform(np.array(x.values).reshape(1, -1)))).argmax()

In [ ]:
model.predict(multi_x)

In [ ]:
dataset.descriptor.keys()

In [ ]:
dataset.descriptor["target"]

In [ ]:
# when
explanation = tabularLore.explain(x)
# then
print(explanation)

In [ ]:
for premise in explanation["rule"]['premises']:
    print(premise["attr"])

In [ ]:
with open("/home/lcorbucci/synth_xai/artifacts/adult/comparison_explanation/lore/lore_1.pkl", "rb") as f:
    shap_explanation = dill.load(f)

In [ ]:
shap_explanation[1]

In [ ]:
shap_explanation[1]["rule"]["premises"]

# Lore Multiclass

In [93]:
from lore_sa.dataset import TabularDataset

# from lore_sa.lorem import LOREM

In [ ]:
class MyModel:
    def __init__(self, model: torch.nn.Module, scaler) -> None:
        self.model = model
        self.scaler = scaler

    def predict(self, x: np.ndarray) -> torch.Tensor:

        x = self.scaler.transform(x)

        predictions = []
        for sample in x:
            sample = torch.Tensor(sample)
            predictions.append(self.model(sample).argmax().item())
        return np.array(predictions)
        # print(x)
        # x = torch.Tensor(x)
        # print(x)
        # return np.array([self.model(sample).argmax().item() for sample in x])
    

seed = 112
current_script_path = Path("./").resolve()
print(current_script_path)
x_train, _, x_test, _, _, _, train_df, test_data = prepare_letter(sweep=False, seed=seed)

# Get the feature names after one-hot encoding
feature_names = list(train_df.drop(columns=["letter"]).columns)

bb_path = "../../../artifacts/letter/bb/letter_BB.pth"
bb = load_bb(bb_path)

bb.to("cpu")


In [95]:
import copy 

train_df_tmp = copy.copy(train_df)
train_df_tmp = train_df_tmp.drop(columns=["letter"])

scaler = MinMaxScaler()
x_train = scaler.fit_transform(train_df_tmp)

In [96]:
model = MyModel(model=bb, scaler=scaler)

num_classes = 2

In [97]:
train_df["letter"] = [int(x) for x in train_df["letter"]]

In [98]:
train_df["letter"] = train_df["letter"].astype('category')

In [ ]:
dataset = TabularDataset.from_dict(train_df, class_name = "letter")
dataset.df.dropna(inplace = True)
dataset.df

In [ ]:
dataset.descriptor.keys()

In [101]:
from lore_sa.bbox import sklearn_classifier_bbox

In [102]:
bbox = sklearn_classifier_bbox.sklearnBBox(model)

In [103]:
from lore_sa.lore import TabularRandomGeneratorLore

tabularLore = TabularRandomGeneratorLore(bbox, dataset)

In [ ]:
num_row = 10
x = dataset.df.iloc[num_row][:-1] # we exclude the target feature
x

In [105]:
# for i in range(0, 100):
#     x = dataset.df.iloc[i][:-1] # we exclude the target feature
#     print(bb(torch.Tensor(scaler.transform(np.array(x.values).reshape(1, -1)))).argmax())

In [106]:
num_rows = 100
multi_x = dataset.df.iloc[:num_row] # we exclude the target feature
multi_x = multi_x.drop(columns = ["letter"])

In [ ]:
bb(torch.Tensor(scaler.transform(np.array(x.values).reshape(1, -1)))).argmax()

In [ ]:
model.predict(multi_x)

In [ ]:
dataset.descriptor.keys()

In [ ]:
dataset.descriptor["target"]

In [ ]:
# when
explanation = tabularLore.explain(x)
# then
print(explanation)

In [ ]:
for premise in explanation["rule"]['premises']:
    print(premise["attr"])

In [113]:
with open("/home/lcorbucci/synth_xai/artifacts/adult/comparison_explanation/lore/lore_1.pkl", "rb") as f:
    shap_explanation = dill.load(f)

In [ ]:
shap_explanation[1]

# Load Shap explanation

In [116]:
with open("/home/lcorbucci/synth_xai/artifacts/adult/comparison_explanation/shap/shap_1.pkl", "rb") as f:
    shap_explanation = dill.load(f)

In [ ]:
shap_explanation[0][0]

In [118]:
class aix_model:
    def __init__(self, model: torch.nn.Module) -> None:
        self.model = model

    def predict(self, x: np.ndarray) -> torch.Tensor:
        x = torch.Tensor(x)
        return self.model(x).argmax(dim=1)

    def predict_proba(self, x: np.ndarray) -> np.ndarray:
        # since the activation function of the last layer is LogSoftmax
        # we need to apply the exponential to the output of the model
        # cast x to be a Tensor
        x = torch.Tensor(x)
        return torch.nn.functional.softmax(self.model(x)).detach().numpy()


In [119]:
seed = 112
current_script_path = Path("./").resolve()

x_train, _, x_test, _, _, _, train_df, test_data = prepare_adult(sweep=False, seed=seed, current_path=current_script_path)
scaler = MinMaxScaler()
_ = scaler.fit_transform(x_train)

# Get the feature names after one-hot encoding
feature_names = list(train_df.drop(columns=["income_binary"]).columns)

# Identify categorical features (before one-hot encoding)
categorical_features = [
    train_df.columns.get_loc(col)
    for col in ["workclass", "education", "marital-status", "occupation", "relationship", "race", "sex", "native-country"]
    if col in train_df.columns
]

# Define the class names
class_names = ["<=50K", ">50K"]

In [120]:
# Initialize LimeTabularExplainer
explainer = LimeTabularExplainer(
    x_train,  # Unscaled training data
    mode="classification",
    feature_names=feature_names,
    categorical_features=categorical_features,
    class_names=class_names,
    discretize_continuous=True,  # Discretize continuous features for better interpretability
)

In [121]:
bb_path = "../../../artifacts/adult/bb/adult_BB.pth"
bb = load_bb(bb_path)

In [122]:
bb.to("cpu")
model = aix_model(model=bb)
# model = bb

In [123]:
def predict_fn(x: np.ndarray) -> np.ndarray:
        model.model.to("cpu")
        prediction = model.predict_proba(x)
        return prediction

In [ ]:
# Generate explanation
sample_idx = 127  # Choose any index from x_test
sample = np.array(x_test[sample_idx])
explanation = explainer.explain_instance(sample, predict_fn, num_features=10)
explanation.as_list()
explanation.local_pred

In [ ]:
explanation.as_list()

In [ ]:
explanation.score

In [ ]:
import re 
feature_names = [feature for feature, weight in explanation.as_list()]
clean_features = [re.sub(r'[<>]=?|\d+(\.\d+)?', '', feature).strip() for feature in feature_names]

clean_features

In [ ]:
store_path = "/home/lcorbucci/synth_xai/artifacts/adult/comparison_explanation/lime/"
file_name = "lime_1.pkl"
store_path = Path(store_path) / file_name
with Path(store_path).open("rb") as f:
    explanation = dill.load(f)


In [ ]:
explanation[45][2]

In [ ]:
store_path = "/home/lcorbucci/synth_xai/artifacts/adult/comparison_explanation/lime/"
file_name = "lime_2.pkl"
store_path = Path(store_path) / file_name
with Path(store_path).open("rb") as f:
    explanation = dill.load(f)


In [ ]:
explanation[45][2]

In [ ]:
store_path = "/home/lcorbucci/synth_xai/artifacts/adult/explanations/"
file_name = "dt_tvae_100000_2500_1.pkl"
store_path = Path(store_path) / file_name
with Path(store_path).open("rb") as f:
    explanation = dill.load(f)


# Shap

In [ ]:
from pathlib import Path

import numpy as np
import torch
import shap

from synth_xai.bb_architectures import MultiClassModel, SimpleModel
from synth_xai.explanations.explanation_utils import load_bb
from synth_xai.utils import prepare_adult
from sklearn.cluster import KMeans

In [ ]:
np.empty((0,2))

In [ ]:
# def get_representative_samples(dataset,samples_per_class=5):
#     '''this function is used to get the representative samples of the dataset. but in a different way.
#     First there is a clustering of all the data. The best k is retrieved using the elbow method.
#     Then, for each cluster we order the samples by the distance from the centroid in 4 bins.
#     Then we take some samples of each bin as the representative sample of the cluster, (in order to have the label)'''
#     # print(dataset.tensors[1].shape,"shape of the labels")
#     num_classes = 2
#     representative_samples = np.empty((0,dataset.shape[1]))
#     labels = np.empty((0,num_classes))
#     # create the kmeans object
#     inertias = []
#     max_K = min(12,dataset.shape[0])
#     for i in range(2,max_K):
#         kmeans = KMeans(n_clusters=i, random_state=42,n_init=3)
#         kmeans.fit(dataset)
#         # get the centroids of the clusters
#         inertias.append(kmeans.inertia_)

#     # coupute the best k
#     inertias_diff = np.diff(inertias)
#     # find the last highest difference
#     k = inertias_diff[::-1].argmax()
#     k = len(inertias) - k
#     print("best k:",k)
#     # plot the elbow plot

#     kmeans = KMeans(n_clusters=k, random_state=42,n_init=3)
#     kmeans.fit(dataset)
#     # get the centroids of the clusters
#     centroids = kmeans.cluster_centers_
#     # get the samples that are mapped to each cluster
#     labels = kmeans.labels_
#     representative_labels = np.empty((0,num_classes))
#     for cluster in range(k):
#         # get the samples of the cluster
#         cluster_samples = dataset[labels==cluster,:]
#         # get the distances of the samples from the centroid
#         distances = np.linalg.norm(cluster_samples-centroids[cluster],axis=1)
#         # order the samples in 4 bins using again KMeans, but on the distances
#         kkkk = min(4,cluster_samples.shape[0])
#         kmeans = KMeans(n_clusters=kkkk, random_state=42,n_init=3)
#         kmeans.fit(distances.reshape(-1,1))
#         ordered_samples = cluster_samples[np.argsort(kmeans.labels_)]

        
#         # divide the samples in 4 bins
#         bins = [ordered_samples[kmeans.labels_==i] for i in range(4)]
#         # plot the number of samples in each bin
    
#         original_labels_of_cluster = dataset[labels==cluster,:]
#         print(original_labels_of_cluster)
        
                                            
#         # take the first sample of each bin, and its relative label
#         for d_i in range(4):
#             # if there are no samples in the bin, skip it
#             if len(bins[d_i]) == 0:
#                 continue
#             # choose a random sample from the bin
#             rnd_sample = 0#np.random.choice(bins[d_i])
#             representative_samples = np.vstack((representative_samples,bins[d_i][rnd_sample]))
#             print(original_labels_of_cluster[rnd_sample,:])
#             print(representative_labels)
#             representative_labels = np.vstack([representative_labels,
#                                                 original_labels_of_cluster[rnd_sample,:]])
                                    
#     print(representative_samples.shape,representative_labels.shape)
#     return representative_samples,representative_labels

In [ ]:
# feats, labs = get_representative_samples(x_train[0:40], samples_per_class=30//num_classes)

In [ ]:
class aix_model:
    def __init__(self, model: torch.nn.Module) -> None:
        self.model = model

    def predict(self, x: np.ndarray) -> torch.Tensor:
        x = torch.Tensor(x)
        return self.model(x).argmax(dim=1)

    def predict_proba(self, x: np.ndarray) -> np.ndarray:
        # since the activation function of the last layer is LogSoftmax
        # we need to apply the exponential to the output of the model
        x = torch.Tensor(x)
        return torch.nn.functional.softmax(self.model(x), dim=1).detach().numpy()

seed = 112
current_script_path = Path("./").resolve()
print(current_script_path)
x_train, _, x_test, _, _, _, train_df, test_data = prepare_adult(sweep=False, seed=seed, current_path=current_script_path)

# Get the feature names after one-hot encoding
feature_names = list(train_df.drop(columns=["income_binary"]).columns)

bb_path = "../../../artifacts/adult/bb/adult_BB.pth"
bb = load_bb(bb_path)

bb.to("cpu")
model = aix_model(model=bb)

num_classes = 2

In [ ]:
# Define SHAP explainer
# explainer = shap.Explainer(model.predict_proba, x_test)
explainer = shap.KernelExplainer(model.predict_proba, data=shap.kmeans(x_test, 20))

In [ ]:

# Generate explanation
sample_idx = 4  # Choose any index from x_test
sample = x_test[sample_idx:sample_idx+1]  # Keep sample as a batch
shap_values = explainer(sample)

In [ ]:
sample

In [ ]:
predicted_class = model.predict(sample)
predicted_class = predicted_class.item()
predicted_class

In [ ]:
# shap.plots.watearfall(shap_values[0,:,predicted_class])

In [ ]:
shap_values.values[0,:,predicted_class]

In [ ]:
feature_importance = shap_values[0,:,predicted_class].values

In [ ]:
feature_importance

In [ ]:
list(zip(feature_names, feature_importance))

In [ ]:
# get the feature names


# Simple example with Adult

In [ ]:
import sklearn
import sklearn.datasets
import sklearn.ensemble
import numpy as np
import lime
import lime.lime_tabular
from __future__ import print_function
np.random.seed(1)

In [ ]:
feature_names = ["Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Marital Status","Occupation", "Relationship", "Race", "Sex", "Capital Gain", "Capital Loss","Hours per week", "Country"]

In [ ]:
data = np.genfromtxt('./adult.data', delimiter=', ', dtype=str)

In [ ]:
labels = data[:,14]
le= sklearn.preprocessing.LabelEncoder()
le.fit(labels)
labels = le.transform(labels)
class_names = le.classes_
data = data[:,:-1]

In [ ]:
categorical_features = [1,3,5, 6,7,8,9,13]

In [ ]:
categorical_names = {}
for feature in categorical_features:
    le = sklearn.preprocessing.LabelEncoder()
    le.fit(data[:, feature])
    data[:, feature] = le.transform(data[:, feature])
    categorical_names[feature] = le.classes_


In [ ]:
data = data.astype(float)

In [ ]:
from sklearn.compose import ColumnTransformer

encoder = ColumnTransformer(
	transformers=[
		('cat', sklearn.preprocessing.OneHotEncoder(), categorical_features)
	],
	remainder='passthrough'
)

In [ ]:
np.random.seed(1)
train, test, labels_train, labels_test = sklearn.model_selection.train_test_split(data, labels, train_size=0.80)


In [ ]:
encoder.fit(data)
encoded_train = encoder.transform(train)

In [ ]:
import xgboost
gbtree = xgboost.XGBClassifier(n_estimators=300, max_depth=5)
gbtree.fit(encoded_train, labels_train)

In [ ]:
sklearn.metrics.accuracy_score(labels_test, gbtree.predict(encoder.transform(test)))

In [ ]:
predict_fn = lambda x: gbtree.predict_proba(encoder.transform(x)).astype(float)

In [ ]:
train

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(train ,feature_names = feature_names,class_names=class_names,
                                                   categorical_features=categorical_features, 
                                                   categorical_names=categorical_names, kernel_width=3)

In [ ]:
np.random.seed(1)
i = 1
exp = explainer.explain_instance(test[i], predict_fn, num_features=5)
exp.show_in_notebook(show_all=False)

In [ ]:
test[i]

In [ ]:
exp.local_pred